In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mutual-funds-and-etfs/ETFs.csv
/kaggle/input/mutual-funds-and-etfs/MutualFund prices - F-K.csv
/kaggle/input/mutual-funds-and-etfs/MutualFund prices - L-P.csv
/kaggle/input/mutual-funds-and-etfs/MutualFund prices - A-E.csv
/kaggle/input/mutual-funds-and-etfs/ETF prices.csv
/kaggle/input/mutual-funds-and-etfs/MutualFunds.csv
/kaggle/input/mutual-funds-and-etfs/MutualFund prices - Q-Z.csv


In [2]:
## Read the file
import pandas as pd
df_funds = pd.read_csv('/kaggle/input/mutual-funds-and-etfs/MutualFunds.csv')
df_funds.head()

,fund_symbol,quote_type,region,fund_short_name,fund_long_name,currency,initial_investment,subsequent_investment,fund_category,fund_family,...,peer_environment_avg,peer_environment_max,social_score,peer_social_min,peer_social_avg,peer_social_max,governance_score,peer_governance_min,peer_governance_avg,peer_governance_max
0,AAAAX,MutualFund,US,DWS RREEF Real Assets Fund - Cl,DWS RREEF Real Assets Fund - Class A,USD,1000.0,50.0,World Allocation,DWS,...,5.05,10.58,7.43,5.98,9.07,11.30,5.43,4.26,7.14,8.11
1,AAAEX,MutualFund,US,AllianzGI Health Sciences Fund,Virtus AllianzGI Health Sciences Fund Class P,USD,1000000.0,NaN,Health,Virtus,...,1.43,3.27,12.96,9.52,12.87,15.08,8.40,4.96,7.68,10.30
2,AAAFX,MutualFund,US,NaN,American Century One Choice Blend+ 2015 Portfo...,USD,2500.0,50.0,Target-Date 2015,American Century Investments,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAAGX,MutualFund,US,Thrivent Large Cap Growth Fund,Thrivent Large Cap Growth Fund Class A,USD,2000.0,50.0,Large Growth,Thrivent Funds,...,2.70,5.81,10.13,7.25,10.14,11.97,8.03,5.30,7.54,8.90
4,AAAHX,MutualFund,US,NaN,American Century One Choice Blend+ 2015 Portfo...,USD,5000000.0,NaN,Target-Date 2015,American Century Investments,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
## print the columns

df_funds.columns.to_list()

['fund_symbol',
 'quote_type',
 'region',
 'fund_short_name',
 'fund_long_name',
 'currency',
 'initial_investment',
 'subsequent_investment',
 'fund_category',
 'fund_family',
 'exchange_code',
 'exchange_name',
 'exchange_timezone',
 'management_name',
 'management_bio',
 'management_start_date',
 'total_net_assets',
 'year_to_date_return',
 'day50_moving_average',
 'day200_moving_average',
 'week52_high_low_change',
 'week52_high_low_change_perc',
 'week52_high',
 'week52_high_change',
 'week52_high_change_perc',
 'week52_low',
 'week52_low_change',
 'week52_low_change_perc',
 'investment_strategy',
 'fund_yield',
 'morningstar_overall_rating',
 'morningstar_risk_rating',
 'inception_date',
 'last_dividend',
 'last_cap_gain',
 'annual_holdings_turnover',
 'investment_type',
 'size_type',
 'fund_annual_report_net_expense_ratio',
 'category_annual_report_net_expense_ratio',
 'fund_prospectus_net_expense_ratio',
 'fund_prospectus_gross_expense_ratio',
 'fund_max_12b1_fee',
 'fund_max_f

In [4]:
#conver file to sql

import sqlite3 as sql
# import sqlalchemy and create a sqlite engine
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table 'playstore' to the sqlite engine
df_funds.to_sql("mutualfunds", con =engine)

In [5]:
## Test Query

sql='''

select * from mutualfunds
limit 1
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,fund_symbol,quote_type,region,fund_short_name,fund_long_name,currency,initial_investment,subsequent_investment,fund_category,...,peer_environment_avg,peer_environment_max,social_score,peer_social_min,peer_social_avg,peer_social_max,governance_score,peer_governance_min,peer_governance_avg,peer_governance_max
0,0,AAAAX,MutualFund,US,DWS RREEF Real Assets Fund - Cl,DWS RREEF Real Assets Fund - Class A,USD,1000.0,50.0,World Allocation,...,5.05,10.58,7.43,5.98,9.07,11.3,5.43,4.26,7.14,8.11


In [6]:
## read the sql file 

## date->biforcate

sql='''

select 
 strftime('%m', inception_date) as Month,
 strftime('%Y', inception_date) as Year,
 category,
 rating,
 return_rating,
 risk_rating
from mutualfunds m
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head() ## 1st 5 rows

OperationalError: (sqlite3.OperationalError) no such column: category
[SQL: 

select 
 strftime('%m', inception_date) as Month,
 strftime('%Y', inception_date) as Year,
 category,
 rating,
 return_rating,
 risk_rating
from mutualfunds m
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
## query 1 :- avg YoY rating for category

##  business inference :- take the avg ratings of each cat, print the category with best rating

## business inferenece:- the one which performs most , the one which performs least

sql='''

select * from (
select 
'Category with maximum Rating :- '||c.category,
round(sum(c.yr_18 +c.yr_19 +c.yr_20 +c.yr_21)/4,2) as avg_YoY

from (
select 
b.category,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.category,
case when a.year='2018' then a.avg_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
 category,
 round(avg(rating),2) as avg_rating
from mutualfunds m
group by category,strftime('%Y', inception_date)
order by category,strftime('%Y', inception_date) desc) a
)b

group by b.category
)c

group by c.category
order by round(sum(c.yr_18 +c.yr_19 +c.yr_20 +c.yr_21)/4,2) desc
limit 1) d

union all

select * from (
select 
'Category with minimum Rating :- '||c.category,
round(sum(c.yr_18 +c.yr_19 +c.yr_20 +c.yr_21)/4,2) as avg_YoY

from (
select 
b.category,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.category,
case when a.year='2018' then a.avg_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
 category,
 round(avg(rating),2) as avg_rating
from mutualfunds m
group by category,strftime('%Y', inception_date)
order by category,strftime('%Y', inception_date) desc) a
)b

group by b.category
)c

group by c.category
order by round(sum(c.yr_18 +c.yr_19 +c.yr_20 +c.yr_21)/4,2) asc
limit 1) d


''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()


In [ ]:
## Query 2:=

## Rating KPIs avg based on  fund_extended_name YoY

## query 1 :- avg YoY rating for category

##  business inference :- take the avg ratings of each cat, print the category with best rating

## business inferenece:- the one which performs most , the one which performs least

## business interence:- too many inconsistenet trends observed in the table with 0 ratings populatd for fund_extend_name

sql='''

select * from (
select
*,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(rating),2) as avg_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_rating desc

''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()


In [ ]:
## Query 3 :- Comparsion table for key rating KPIs YoY

## Business Inference:- Terms

## Rating:- Performance of the package
## risk rating:- degree of risk
## return rating:-  degreee of output

sql='''

select
rating.fund_extended_name,
rating.avg_rating,
return.avg_return_rating,
risk.avg_risk_rating
from (
select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(rating),2) as avg_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_rating desc
) rating
join 
(select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_return_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_return_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_return_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_return_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_return_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(return_rating),2) as avg_return_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_return_rating desc
) return on rating.fund_extended_name=return.fund_extended_name
join 
(select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_risk_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_risk_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_risk_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_risk_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_risk_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(risk_rating),2) as avg_risk_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_risk_rating desc

) risk on rating.fund_extended_name=risk.fund_extended_name


order by risk.avg_risk_rating desc
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head(20)


In [ ]:
## Query 3 :- Fund extended name vs performance groups and riskvsreturnmatrix

## Business Inference:- Terms

## the goal is to idenfity groups and help business team understand performance trends 

## Rating:- Performance of the package
## risk rating:- degree of risk
## return rating:-  degreee of output

sql='''
select
f.fund_extended_name,
f.Performance,
f.RiskvsReturnMatrix
from (
select
e.row_number,
e.fund_extended_name,
case when e.avg_rating > avg(e.avg_rating)  over (order by e.row_number asc) then 'Performing fund' else 'Non-Performing fund' end as Performance,
case 
when e.avg_risk_rating > avg(e.avg_risk_rating) over (order by e.row_number asc) and e.avg_return_rating > avg(e.avg_return_rating) over (order by e.row_number asc) then 'High-Risk and High-Return'
when e.avg_risk_rating >= avg(e.avg_risk_rating) over (order by e.row_number asc) and e.avg_return_rating <= avg(e.avg_return_rating) over (order by e.row_number asc) then 'High-Risk and Low-Return'
when e.avg_risk_rating < avg(e.avg_risk_rating) over (order by e.row_number asc) and e.avg_return_rating < avg(e.avg_return_rating) over (order by e.row_number asc) then 'Low-Risk and Low-Return'
else 'Low-Risk and High-Return' end as RiskvsReturnMatrix
from (
select
ROW_NUMBER() OVER (ORDER BY rating.fund_extended_name asc) as row_number,
rating.fund_extended_name,
rating.avg_rating,
return.avg_return_rating,
round(risk.avg_risk_rating,2) as avg_risk_rating
from (
select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(rating),2) as avg_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_rating desc
) rating
join 
(select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_return_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_return_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_return_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_return_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_return_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(return_rating),2) as avg_return_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_return_rating desc
) return on rating.fund_extended_name=return.fund_extended_name
join 
(select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_risk_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_risk_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_risk_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_risk_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_risk_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(risk_rating),2) as avg_risk_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_risk_rating desc

) risk on rating.fund_extended_name=risk.fund_extended_name

)e

group by e.row_number) f
group by 1
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head(20)


In [ ]:
## Query 4 :- Performance and RiskvsReturn Matrix analysis for business review


sql='''
select
'The Comparision is as follows :- '||
max(h.Performing_fund) as Value_one,
max(h.Non_Performing_fund) as Value_two
from (
select 
g.Performance,
sum(case when g.performance='Performing fund' then 1 else 0 end) as Performing_fund,
sum(case when g.performance='Non-Performing fund' then 1 else 0 end) as Non_Performing_fund
from (
select
f.fund_extended_name,
f.Performance,
f.RiskvsReturnMatrix
from (
select
e.row_number,
e.fund_extended_name,
case when e.avg_rating > avg(e.avg_rating)  over (order by e.row_number asc) then 'Performing fund' else 'Non-Performing fund' end as Performance,
case 
when e.avg_risk_rating > avg(e.avg_risk_rating) over (order by e.row_number asc) and e.avg_return_rating > avg(e.avg_return_rating) over (order by e.row_number asc) then 'High-Risk and High-Return'
when e.avg_risk_rating >= avg(e.avg_risk_rating) over (order by e.row_number asc) and e.avg_return_rating <= avg(e.avg_return_rating) over (order by e.row_number asc) then 'High-Risk and Low-Return'
when e.avg_risk_rating < avg(e.avg_risk_rating) over (order by e.row_number asc) and e.avg_return_rating < avg(e.avg_return_rating) over (order by e.row_number asc) then 'Low-Risk and Low-Return'
else 'Low-Risk and High-Return' end as RiskvsReturnMatrix
from (
select
ROW_NUMBER() OVER (ORDER BY rating.fund_extended_name asc) as row_number,
rating.fund_extended_name,
rating.avg_rating,
return.avg_return_rating,
round(risk.avg_risk_rating,2) as avg_risk_rating
from (
select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(rating),2) as avg_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_rating desc
) rating
join 
(select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_return_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_return_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_return_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_return_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_return_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(return_rating),2) as avg_return_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_return_rating desc
) return on rating.fund_extended_name=return.fund_extended_name
join 
(select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_risk_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_risk_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_risk_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_risk_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_risk_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(risk_rating),2) as avg_risk_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_risk_rating desc

) risk on rating.fund_extended_name=risk.fund_extended_name

)e

group by e.row_number) f
group by 1
) g

group by 1
) h
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()


In [ ]:
## Query 5 :- RiskvsReturn Matrix analysis for business review

## business inference:- Top groups are low-low and high-high

## checkpoint:- We need to access on count of low-low (for example how much is low for an average customer)


sql='''
select
'The Comparision is as follows :- '||
max(h.High_High) as High_High,
max(h.Low_High) as Low_High,
max(h.High_Low) as High_Low,
max(h.Low_Low) as Low_Low
from (
select 
g.RiskvsReturnMatrix,
sum(case when g.RiskvsReturnMatrix='High-Risk and High-Return' then 1 else 0 end) as High_High,
sum(case when g.RiskvsReturnMatrix='Low-Risk and High-Return' then 1 else 0 end) as Low_High,
sum(case when g.RiskvsReturnMatrix='High-Risk and Low-Return' then 1 else 0 end) as High_Low,
sum(case when g.RiskvsReturnMatrix='Low-Risk and Low-Return' then 1 else 0 end) as Low_Low
from (
select
f.fund_extended_name,
f.Performance,
f.RiskvsReturnMatrix
from (
select
e.row_number,
e.fund_extended_name,
case when e.avg_rating > avg(e.avg_rating)  over (order by e.row_number asc) then 'Performing fund' else 'Non-Performing fund' end as Performance,
case 
when e.avg_risk_rating > avg(e.avg_risk_rating) over (order by e.row_number asc) and e.avg_return_rating > avg(e.avg_return_rating) over (order by e.row_number asc) then 'High-Risk and High-Return'
when e.avg_risk_rating >= avg(e.avg_risk_rating) over (order by e.row_number asc) and e.avg_return_rating <= avg(e.avg_return_rating) over (order by e.row_number asc) then 'High-Risk and Low-Return'
when e.avg_risk_rating < avg(e.avg_risk_rating) over (order by e.row_number asc) and e.avg_return_rating < avg(e.avg_return_rating) over (order by e.row_number asc) then 'Low-Risk and Low-Return'
else 'Low-Risk and High-Return' end as RiskvsReturnMatrix
from (
select
ROW_NUMBER() OVER (ORDER BY rating.fund_extended_name asc) as row_number,
rating.fund_extended_name,
rating.avg_rating,
return.avg_return_rating,
round(risk.avg_risk_rating,2) as avg_risk_rating
from (
select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(rating),2) as avg_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_rating desc
) rating
join 
(select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_return_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_return_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_return_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_return_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_return_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(return_rating),2) as avg_return_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_return_rating desc
) return on rating.fund_extended_name=return.fund_extended_name
join 
(select * from (
select
c.fund_extended_name,
sum(c.yr_18 +c.yr_19 + c.yr_20 + c.yr_21)/4 as avg_risk_rating
from (
select 
b.fund_extended_name,
max(b.yr_2018) as yr_18,
max(b.yr_2019) as yr_19,
max(b.yr_2020) as yr_20,
max(b.yr_2021) as yr_21

from (
select
a.fund_extended_name,
case when a.year='2018' then a.avg_risk_rating else 0 end as yr_2018,
case when a.year='2019' then a.avg_risk_rating else 0 end as yr_2019,
case when a.year='2020' then a.avg_risk_rating else 0 end as yr_2020,
case when a.year='2021' then a.avg_risk_rating else 0 end as yr_2021

from

(select 
 strftime('%Y', inception_date) as Year,
  fund_extended_name,
 round(avg(risk_rating),2) as avg_risk_rating
from mutualfunds m
group by fund_extended_name,strftime('%Y', inception_date)
order by fund_extended_name,strftime('%Y', inception_date) desc) a
)b

group by b. fund_extended_name
) c

group by c.fund_extended_name 
) d
order by d.avg_risk_rating desc

) risk on rating.fund_extended_name=risk.fund_extended_name

)e

group by e.row_number) f
group by 1
) g

group by 1
) h
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()


In [ ]:
## Query 6 :- identify  fund_extended_name, category, investment_type, size_type trends for turn overs

## Top turnover fund_extend_name:- AXS Market Neutral Fund Investor Class
## Top turnover category:- Intermediate Core-Plus Bond
## Top turnover investment_type:- Blend
## Top turnover size_type:- Large






sql='''
select
'MAX turnover by fund_extend_name :- ' as Category_Name, a.fund_extended_name as title,
max(a.holdings_turnover) as value

from
(select 
row_number () over (order by fund_extended_name,category,investment_type,size_type asc) as row_number,
fund_extended_name,
category,
investment_type,
size_type,
sum(holdings_turnover) as holdings_turnover
from mutualfunds m
group by fund_extended_name) a

union all

select
'MAX turnover by category :- 'as Category_Name, a.category as title,
max(a.holdings_turnover) as value

from
(select 
row_number () over (order by fund_extended_name,category,investment_type,size_type asc) as row_number,
fund_extended_name,
category,
investment_type,
size_type,
sum(holdings_turnover) as holdings_turnover
from mutualfunds m
group by category) a

union all

select
'MAX turnover by investment_type :- 'as Category_Name, a.investment_type as title,
max(a.holdings_turnover) as value

from
(select 
row_number () over (order by fund_extended_name,category,investment_type,size_type asc) as row_number,
fund_extended_name,
category,
investment_type,
size_type,
sum(holdings_turnover) as holdings_turnover
from mutualfunds m
group by investment_type) a

union all

select
'MAX turnover by size_type :- ' as Category_Name, a.size_type as title,
max(a.holdings_turnover) as value

from
(select 
row_number () over (order by fund_extended_name,category,investment_type,size_type asc) as row_number,
fund_extended_name,
category,
investment_type,
size_type,
sum(holdings_turnover) as holdings_turnover
from mutualfunds m
group by size_type) a

''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()


In [ ]:
## Query 7 :- 







sql='''
select 

round(avg(fund_expense_projection_3y),2) as avg_expe_threeyrs,
round(avg(fund_expense_projection_5y),2) as avg_expe_fiveyrs,
round(avg(fund_expense_projection_10y),2) as avg_expe_tenyrs
from mutualfunds m
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()


In [ ]:
## Query 7 :- Expense_Project_Trend

sql='''
with expense as (
select 
m.fund_family,
sum(case when m.fund_expense_projection_3y > 481.1 then 1 else 0 end) as Expense_Project_Trend_3y,
sum(case when m.fund_expense_projection_5y > 758.82 then 1 else 0 end) as Expense_Project_Trend_5y,
sum(case when m.fund_expense_projection_10y > 1557.44 then 1 else 0 end) as Expense_Project_Trend_10y 
from mutualfunds m
group by 1)

select * from (
select
a.Expense_Year,
max(a.thitreenDActivist) as thitreenDActivist,
max(a.threesixtyoneFunds) as threesixtyoneFunds,
max(a.AAM) as AAM,
max(a.AAMA) as AAMA,
max(a.ABR) as ABR
from

(select
'03 years' as Expense_Year,
case when e.fund_family='13D Activist Fund' then Expense_Project_Trend_3y else 0 end as thitreenDActivist,
case when e.fund_family='361 Funds' then Expense_Project_Trend_3y else 0 end as threesixtyoneFunds,
case when e.fund_family='AAM' then Expense_Project_Trend_3y else 0 end as AAM,
case when e.fund_family='AAMA' then Expense_Project_Trend_3y else 0 end as AAMA,
case when e.fund_family='ABR' then Expense_Project_Trend_3y else 0 end as ABR

from expense e
) a

union

select
d.Expense_Year,
max(d.thitreenDActivist) as thitreenDActivist,
max(d.threesixtyoneFunds) as threesixtyoneFunds,
max(d.AAM) as AAM,
max(d.AAMA) as AAMA,
max(d.ABR) as ABR
from

(select
'05 years' as Expense_Year,
case when e.fund_family='13D Activist Fund' then Expense_Project_Trend_5y else 0 end as thitreenDActivist,
case when e.fund_family='361 Funds' then Expense_Project_Trend_5y else 0 end as threesixtyoneFunds,
case when e.fund_family='AAM' then Expense_Project_Trend_5y else 0 end as AAM,
case when e.fund_family='AAMA' then Expense_Project_Trend_5y else 0 end as AAMA,
case when e.fund_family='ABR' then Expense_Project_Trend_5y else 0 end as ABR

from expense e
) d

union 

select
c.Expense_Year,
max(c.thitreenDActivist) as thitreenDActivist,
max(c.threesixtyoneFunds) as threesixtyoneFunds,
max(c.AAM) as AAM,
max(c.AAMA) as AAMA,
max(c.ABR) as ABR
from

(select
'10 years' as Expense_Year,
case when e.fund_family='13D Activist Fund' then Expense_Project_Trend_10y else 0 end as thitreenDActivist,
case when e.fund_family='361 Funds' then Expense_Project_Trend_10y else 0 end as threesixtyoneFunds,
case when e.fund_family='AAM' then Expense_Project_Trend_10y else 0 end as AAM,
case when e.fund_family='AAMA' then Expense_Project_Trend_10y else 0 end as AAMA,
case when e.fund_family='ABR' then Expense_Project_Trend_10y else 0 end as ABR

from expense e
) c
) f

order by f.Expense_Year asc
''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()


In [ ]:
## Query 7 :- Expense_Project_Trend

sql='''

select
'fund expense' as fund_expense_YoY,
m.total_funds_2019,
a.total_funds_2020
from (
select
1 as col,
'2019' as Year,
sum(fund_return_2019_q1+  fund_return_2019_q2 + fund_return_2019_q3 +  fund_return_2019_q4) as total_funds_2019
from mutualfunds) m

join (
select * from (
select
1 as col,
'2020' as Year,
sum(fund_return_2020_q1+  fund_return_2020_q2 + fund_return_2020_q3 +  fund_return_2020_q4) as total_funds_2020
from mutualfunds) b

) a on a.col=m.col

''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()


In [ ]:
## Query 7 :- Expense_Project_Trend

sql='''


select 
'Expense' as YoY,
b.total_funds_2019,
a.total_funds_2020
from (
select
1 as col,
'2019' as Year,
sum(fund_return_2019_q1+  fund_return_2019_q2 + fund_return_2019_q3 +  fund_return_2019_q4) as total_funds_2019
from mutualfunds m
) b

join (
select * from 
(select
1 as col,
'2020' as Year,
sum(fund_return_2020_q1+  fund_return_2020_q2 + fund_return_2020_q3 +  fund_return_2020_q4) as total_funds_2020
from mutualfunds m)


) a on b.col=a.col


''';

    
df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()
